# Multi-Commodity Flow

**Выполнила: Ковалева Варвара**

## Постановка задачи

### Дано:
- Ориентированный граф $ G = (V, E) $
- Множество товаров (commodities) $ K $
- Для каждого товара $ k \in K $: источник $ s_k $, сток $ t_k $, требуемый поток $ d_k $
- Для каждого ребра $ e \in E $: стоимость использования одним ТС $ c_e $, вместимость ТС $ C $
- Для каждого узла $ v \in V $: $ f_v \in \mathbb{R}_+ $ — стоимость перегруза единицы груза в узле $ v $, $ W_v \in \mathbb{R}_+ $ — максимальное количество перегруза в узле $ v $
- Для каждого товара $ k \in K $: $ \Pi_k $ — множество всех возможных путей из $ s_k $ в $ t_k $ в графе $ G $

---

### Переменные:
- $ x^k_p \in \mathbb{R}_+ $ — поток товара $ k $ по пути $ p \in \Pi_k $
- $ y_e \in \mathbb{Z}_+ $ — количество ТС, используемых на ребре $ e \in E $

---

### Целевая функция:
$$
\min \left(
\sum_{e \in E} c_e \cdot y_e +
\sum_{k \in K} \sum_{p \in \Pi_k}  \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}}  f_v \cdot x^k_p
\right)
$$

---

### Ограничения:

1. **Полный поток для каждого товара:**
   $$
   \sum_{p \in \Pi_k} x^k_p = d_k \quad \forall k \in K
   $$
   *Обеспечивает удовлетворение спроса для каждого товара.*

2. **Связь между потоком и количеством ТС:**
   $$
   \sum_{k \in K} \sum_{p \in \Pi_k : e \in p} x^k_p \leq C \cdot y_e \quad \forall e \in E, \quad y_e \in \mathbb{Z}_+
   $$
   *Количество ТС $ y_e $ определяется суммарным потоком на ребре $ e $.*

3. **Неотрицательность потоков:**
   $$
   x^k_p \geq 0 \quad \forall k \in K, \, \forall p \in \Pi_k
   $$

4. **Максимальная перегрузка в узле**
   $$
   \sum_{k \in K} \sum_{p \in \Pi_k}  \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}} x^k_p \le W_e \quad \forall e \in E
   $$

In [25]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import time

! pip install gurobipy -q

import gurobipy as gp
from gurobipy import GRB

## Подход к решению

В кратце алгоритм можно разбить на два этапа:
1. Поиск результата без учета огрничения по максимальной перегрузке в узле
2. Разгрузка перегруженных узлов с помощью итеративного исключения их из графа

### 1. Расчёт результата без учета максимальной перегрузки в узлах

После считывания данных и создания графа в виде списка ребер с стоимостями, модифицируем немного это список следующим образом: для каждого ребра добавим стоимости перегруза его концов. Это нужно для того, чтобы учесть стоимость перегруза в поиске крачайших путей.

Собственно, следующий шаг и есть поиск k кратчайших путей, где k вычисляется по определнной формуле в зависимости от вместимости транспортного средства и объема товара.

Затем с помощью солвера находим решение в путевой постановке задачи с использование найденных k кратчайших путей.

In [26]:
class Data: # Общие данные задачи
  def __init__(self, vehicle_capacity, folder):
    self.offices_ids = []
    self.products_ids = []
    self.edges = []
    self.offices = {}
    self.products = {}
    self.edges_price = {}
    self.vehicle_capacity = vehicle_capacity

    df_offices = pd.read_csv(folder+'offices.csv', usecols=['office_id','transfer_price','transfer_max'], dtype={'office_id':int,'transfer_price':float, 'transfer_max':int})
    df_reqs = pd.read_csv(folder+'reqs.csv', dtype={'src_office_id':int,'dst_office_id':int,'volume':int})
    df_distance_matrix = pd.read_csv(folder+'distance_matrix.csv', usecols =['src','dst','price'], dtype={'src':int,'dst':int, 'price':float})

    for(i, row) in df_offices.iterrows():
      self.offices_ids.append(row['office_id'])
      self.offices[row['office_id']] = (row['transfer_price'], row['transfer_max'])

    for(i, row) in df_reqs.iterrows():
      self.products_ids.append(i)
      self.products[i] = (row['src_office_id'], row['dst_office_id'], row['volume'])

    for(i, row) in df_distance_matrix.iterrows():
      if(row['src'] != row['dst'] and row['price']>0):
        edge = (row['src'], row['dst'])
        self.edges.append(edge)
        self.edges_price[edge] = row['price']


In [27]:
class SolverData: # Данные, необходимые для солвера
  def __init__(self):
    self.product_path_ids = [] # массив (индекс товара - индекс пути)
    self.product_path_count = {} # товар - количество путей
    self.paths = {} # рассматриваемые пути продуктов
    self.flow_lb = []
    self.flow_ub = []

In [28]:
def count_price_of_edjes(edges_price, offices): # Расчет стоимостей ребер с учетом стоимостей перегруза в вершинах
  edges_with_transfer_price = edges_price.copy()
  for edge in edges_price:
    edges_with_transfer_price[edge] += offices[edge[0]][0]+offices[edge[1]][0]
  return edges_with_transfer_price

In [29]:
def edges_to_nx_graph(edges_price):
    G = nx.DiGraph()
    for edge, price in edges_price.items():
      G.add_edge(edge[0], edge[1], weight = price)
    return G

Хоть мы и прибавили к каждому ребру стоимость перегрузки в его вершинах, стоит не забывать, что мы не учитываем эту стоимость в source и target вершинах. Поэтому в следующем методе как раз произходит вычитание их стоимостей из смежных с ними ребер.

In [30]:
def remove_transfer_price_source_target(edges_with_transfer_price, source, target, offices):
  edges_for_product = edges_with_transfer_price.copy()

  price_source = offices[source][0]
  price_target = offices[target][0]

  prices = [price_source, price_source, price_target, price_target]

  for v in offices:
    edges = [(source, v), (v, source), (target, v), (v, target)]
    for i in range(4):
      if(edges[i] in edges_for_product):
        edges_for_product[edges[i]] = edges_for_product[edges[i]] - prices[i]

  return edges_for_product

In [31]:
def calc_product_paths(edges_with_transfer_price, source, target, k, office): # Расчёт k-кратчайших путей для конкретного товара
  edges_for_product = remove_transfer_price_source_target(edges_with_transfer_price, source, target, office)

  graph = edges_to_nx_graph(edges_for_product)
  paths_gen = nx.shortest_simple_paths(graph, source=source, target=target, weight='weight')
  paths_list = list(itertools.islice(paths_gen, k))

  return paths_list


In [32]:
def calc_paths(edges_with_transfer_price, data): # Расчёт кратчайших путей для всех товаров
  paths = {}

  for product, product_data in data.products.items():
    count_of_paths = product_data[2]//(data.vehicle_capacity)+3
    paths[product] = calc_product_paths(edges_with_transfer_price, product_data[0], product_data[1], count_of_paths, data.offices)

  return paths

In [33]:
def make_solver_data(paths, products): # Формирование данных для солвера
  solver_data = SolverData()

  for product, product_paths in paths.items():
    solver_data.product_path_count[product] = len(product_paths)
    for i, path in enumerate(product_paths):
      solver_data.product_path_ids.append((product, i))
      solver_data.paths[(product, i)] = path

  max_ub = -1
  for product, product_data in products.items():
    max_ub = max(max_ub, product_data[2])
  solver_data.flow_lb = [0]*len(solver_data.product_path_ids)
  solver_data.flow_ub = [max_ub]*len(solver_data.product_path_ids)

  return solver_data

In [34]:
def solve_min_cost_flow(data, solver_data): # Реализация солвера без учета ограничения на максимальный вес перегрузки

  model = gp.Model('MinCostFlow')

  # Переменные
  x = model.addVars(solver_data.product_path_ids, name='flow', lb=solver_data.flow_lb, ub = solver_data.flow_ub, vtype = GRB.INTEGER)
  y = model.addVars(data.edges, name='vehicles count', lb=0, vtype = GRB.INTEGER)

  # Целевая функция
  model.setObjective(
      gp.quicksum(data.edges_price[e]*y[e] for e in data.edges) + gp.quicksum(gp.quicksum(data.offices[v][0]*x[x_i_p] for v in solver_data.paths[x_i_p][1:-1]) for x_i_p in solver_data.product_path_ids),
        GRB.MINIMIZE
    )

  # Ограничения

  for product in data.products_ids:
    model.addConstr(gp.quicksum(x[(product,p_i)] for p_i in range(solver_data.product_path_count[product])) == data.products[product][2])


  edges_paths = {edge: set() for edge in data.edges}
  for product_path_ind, path in solver_data.paths.items():
    for i in range(len(path)-1):
      edges_paths[(path[i], path[i+1])].add(product_path_ind)

  for e in data.edges:
    model.addConstr(gp.quicksum(x[x_i_p] for x_i_p in edges_paths[e]) <= data.vehicle_capacity*y[e])

  # Решение
  model.setParam('OutputFlag', 0)
  start_time = time.time()
  model.optimize()
  sol_time = time.time() - start_time

  if model.status == GRB.OPTIMAL:
    flow = {product_path_id: x[product_path_id].x for product_path_id in solver_data.product_path_ids if x[product_path_id].x>0}
    v_count = {e: y[e].x for e in data.edges}
    obj_val = model.objVal
    return flow, v_count ,obj_val
  else:
    return None, None, None

## 2. Разгрузка перегруженых узлов

На данном этапе происходит процесс разгрузки перегруженных узлов.

Первым делом находим перегруженные узлы, а также пути, которые через эти узлы проходят. Затем сортируем эти пути по убыванию веса. Берем самый большой результат и находим маскимальный вес данного пути, который неоходим для того, чтобы все загруженные на этом пути узлы пришли в норму по объему перегрузки (объема этого пути может не хватить, тогда убираем этот путь вовсе). Продолжаем с следющем по величине пути пока все узлы не разгрузятся. Таким образом, мы получили множество товаров, некторые пути которых были исключены из расчета и их необходимо перераспределить.

Далее уже на графе, из которого убрали перегруженные ребра, находим k-кратчайших путей для этих исключенных товаров. Для всех остальных товаров и путей, не создающих перегруза вершин, оставляем найденный на прошлом этапе решение. Грубо говоря, на данном этапе мы перестраиваем некторые пути так, чтобы избавится от перегруза, созданных в узлах.

Однако данный процесс не исключает того, что у нас могут появится новые перегруженные узлы. Для этого мы снова повторяем данную процедуру на графе с уже исключенными ранее вершинами.

В любом случае, если граммотно заданы условия задачи, мы придем к ситуации, когда будет исключенно достаточно вершин и ни одна из оставшихся вершин не будет перегруженна.

In [35]:
class OverloadData: # Данные, расчитываемые при перегрузке вершин
  def __init__(self):
    self.flow = {}
    self.overload_office = set()
    self.overload_product = set()
    self.overload_paths = set()

In [36]:
def calc_overloads(data, solver_data, flow): #Расчитать необходимые данные для процесса разгрузки
  overloads = OverloadData()

  overload_office_weight = {office: 0 for office in data.offices_ids}
  path_offices = {product_path_id: set() for product_path_id in solver_data.product_path_ids}
  office_paths = {office: set() for office in data.offices_ids}
  overloads.flow = flow.copy()

  for product_path_id, weight in flow.items():
    for office in solver_data.paths[product_path_id][1:-1]:
      overload_office_weight[office] += weight
      office_paths[office].add(product_path_id)

  for office, weight in overload_office_weight.items():
    if weight > data.offices[office][1]:
      overloads.overload_office.add(office)
      overloads.overload_paths.update(office_paths[office])

  sorted_paths = sorted(overloads.overload_paths, key=lambda x: flow[x], reverse=True)

  over_office = overloads.overload_office.copy()
  while len(over_office)>0:
    path_to_delete = sorted_paths[0]
    sorted_paths.pop(0)
    max_weight = 0;
    for office in solver_data.paths[path_to_delete][1:-1]:
      max_weight = max(max_weight, min(overload_office_weight[office]-data.offices[office][1],flow[path_to_delete]))
    for office in solver_data.paths[path_to_delete][1:-1]:
      overload_office_weight[office] -= max_weight
      if (office in over_office and overload_office_weight[office] <= data.offices[office][1]):
        over_office.remove(office)
    if(overloads.flow[path_to_delete]<=max_weight):
      overloads.flow.pop(path_to_delete)
    else:
      overloads.flow[path_to_delete] -= max_weight
    overloads.overload_product.add(path_to_delete[0])

  return overloads


In [37]:
def find_edges_with_overload_offices(edges_with_transfer_price, data, overload_offices): # Найти ребра, которые необходимо исключить в связи с исключением вершины графа
  edges_without_overload = edges_with_transfer_price.copy()
  edges_with_overload_offices = set()
  for edge in edges_with_transfer_price:
    if edge[0] in overload_offices or edge[1] in overload_offices:
      edges_without_overload.pop(edge)
      edges_with_overload_offices.add((edge, data.edges_price[edge]))

  return edges_without_overload, edges_with_overload_offices

In [38]:
def restore_overlaod_offices(edges_without_overload, edges_with_overload_offices, source, target, overload_offices, data): # Восстановить часть ребер, связанных с sorce И target товара
  edges_for_product = edges_without_overload.copy()
  for (edge, price) in edges_with_overload_offices:
    price = data.edges_price[edge]
    if ((edge[0] == source or edge[0] == target) and (edge[1] == source or edge[1] == target)):
      edges_for_product[edge] = price
    elif(edge[0] == source and edge[0] == target and edge[1] not in overload_offices):
      edges_for_product[edge] = price
    elif(edge[1] == source and edge[1] == target and edge[0] not in overload_offices):
      edges_for_product[edge] = price

  return edges_for_product

In [39]:
def calc_paths_overload(overloads, edges_with_transfer_price, data, solver_data): # Расчет путей при разргрузке
  paths = {product : [] for product in data.products_ids}

  edges_without_overload, edges_with_overload_offices = find_edges_with_overload_offices(edges_with_transfer_price, data, overloads.overload_office)
  for product in overloads.overload_product:
    product_data = data.products[product]
    count_of_paths = product_data[2]//(data.vehicle_capacity//3)+3
    edges_for_product = restore_overlaod_offices(edges_without_overload, edges_with_overload_offices, product_data[0], product_data[1], overloads.overload_office, data)
    paths[product] = calc_product_paths(edges_for_product, product_data[0], product_data[1], count_of_paths, data.offices)

  for (product, path_id) in overloads.flow :
    if(not(product in overloads.overload_product and (product, path_id) not in overloads.overload_paths)):
      paths[product].append(solver_data.paths[(product, path_id)])

  return paths

In [40]:
def get_string_from_path(path): # Вспомогательны метод для красивово вывода в файл
  string = '('
  for i in range(len(path)-1):
    string += str(int(path[i]))+', '
  string += str(int(path[-1]))+')'
  return string

In [41]:
def write_result_into_file(file, flow, v_count , obj_val, sol_time, solver_data): # Запись результата в файл
  with open(file, 'w') as f:
    f.write('product, path, volume\n')
    for product_path_id, weight in flow.items():
      f.write(str(product_path_id[0])+', '+get_string_from_path(solver_data.paths[product_path_id])+', '+str(weight)+'\n')
    f.write('\nedge, vehicle_count\n')
    for edge, count in v_count.items():
      if(count>0):
        f.write('('+str(int(edge[0]))+', '+str(int(edge[1]))+'), '+str(count)+'\n')
    f.write('\nResult: '+str(obj_val)+'\n')
    f.write('\nSolution time: '+str(sol_time)+'\n')

Следующий метод и реализует весь процесс решения задачи

In [42]:
def solve_MCF(vehicle_capacity, folder, output_file):
  start_time = time.time()
  data = Data(vehicle_capacity,folder)
  edges_with_transfer_price = count_price_of_edjes(data.edges_price, data.offices)
  paths = calc_paths(edges_with_transfer_price, data)
  solver_data = make_solver_data(paths, data.products)

  step=1;
  while (True):
    flow, v_count, obj_val = solve_min_cost_flow(data, solver_data)

    if(flow == None):
      print ("Грустная история(")
      break

    print (step,". ", obj_val)
    step+=1

    overloads = calc_overloads(data, solver_data, flow)
    if (len(overloads.overload_office) == 0):
      break
    paths = calc_paths_overload(overloads, edges_with_transfer_price, data, solver_data)
    solver_data = make_solver_data(paths, data.products)
    for i, product_path_id in enumerate(solver_data.product_path_ids):
      if(product_path_id in overloads.overload_paths and product_path_id in overloads.flow):
        solver_data.flow_ub[i] = overloads.flow[product_path_id]
        solver_data.flow_lb[i] = overloads.flow[product_path_id]
  sol_time = time.time() - start_time
  if(flow != None):
    write_result_into_file(output_file, flow, v_count , obj_val, sol_time, solver_data)
    print('Result: '+str(obj_val)+'\n')
    print('Solution time: '+str(sol_time)+'\n')


## Результаты

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
solve_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/10_nodes/', '10_nodes.txt')

1 .  1060958962.6899999
2 .  1760538002.9975998
3 .  1997429082.1576
4 .  1640894604.7452002
5 .  1856440028.7975998
6 .  1689706846.5976
7 .  1601942765.9975998
8 .  1800889275.8968
9 .  1674410063.1452
10 .  1866934815.1976
Result: 1866934815.1976

Solution time: 0.3382556438446045



С дата-сетов в 20 вершин почему-то не отрабатвыает даже часть без ограничения на максимальную перегрузку

In [45]:
solve_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/20_nodes/', '20_nodes.txt')

Грустная история(


А при попытке протестировать датасет с 50 и 140 вершинами столкнулась с ограничениями лиценции солвера ☹

In [46]:
solve_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/50_nodes/', '50_nodes.txt')

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
solve_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/140_nodes/', '140_nodes.txt')

## Точное решение

Здесь я добавила решение задачи точным образом с помощью потоковой постановки задачи

In [47]:
class OptimSolverData:
  def __init__(self):
    self.product_edge_ids = [] # массив (индекс товара - индекс ребра)
    self.edge_in = {}
    self.edge_out = {}

In [48]:
def solve_optim_MCF(vehicle_capacity, folder):
  start_time = time.time()
  data = Data(vehicle_capacity, folder)

  optim_solver_data = OptimSolverData()
  optim_solver_data.edge_in = {office: set() for office in data.offices_ids}
  optim_solver_data.edge_out = {office: set() for office in data.offices_ids}
  for edge in data.edges:
    for product in data.products_ids:
      optim_solver_data.product_edge_ids.append((edge, product))
    for office in data.offices_ids:
      optim_solver_data.edge_out[edge[0]].add(edge)
      optim_solver_data.edge_in[edge[1]].add(edge)

  flow, v_count , obj_val = ground_truth_solve_min_cost_flow(data, optim_solver_data)
  sol_time = time.time() - start_time
  print('Result: '+str(obj_val)+'\n')
  print('Solution time: '+str(sol_time)+'\n')


In [49]:
def ground_truth_solve_min_cost_flow(data, solver_data):

  model = gp.Model('MinCostFlow')

  # Переменные
  x = model.addVars(solver_data.product_edge_ids, name='flow', vtype = GRB.INTEGER)
  y = model.addVars(data.edges, name='vehicles count', lb=0, vtype = GRB.INTEGER)

  # Целевая функция
  model.setObjective(
      gp.quicksum(data.edges_price[e]*y[e] for e in data.edges) + gp.quicksum(data.offices[office][0] * gp.quicksum(x[x_i_p] for x_i_p in solver_data.product_edge_ids if data.products[x_i_p[1]][0] != office and data.products[x_i_p[1]][1]!= office and x_i_p[0] in solver_data.edge_in[office]) for office in data.offices_ids),
        GRB.MINIMIZE
    )

  # Ограничения

  for product in data.products_ids:
    for office in data.offices_ids:
      if (office == data.products[product][0]):
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == data.products[product][2])
      elif (office == data.products[product][1]):
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == -data.products[product][2])
      else:
        model.addConstr(gp.quicksum(x[(edge, product)] for edge in solver_data.edge_out[office]) - gp.quicksum(x[(edge, product)] for edge in solver_data.edge_in[office]) == 0)


  for edge in data.edges:
    model.addConstr(gp.quicksum(x[(edge, product)] for product in data.products_ids) <= data.vehicle_capacity*y[edge])

  for office in data.offices_ids:
    model.addConstr(gp.quicksum(x[x_i_p] for x_i_p in solver_data.product_edge_ids if data.products[x_i_p[1]][0] != office and data.products[x_i_p[1]][1]!= office and x_i_p[0] in solver_data.edge_in[office])<= data.offices[office][1])

  # Решение
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    flow = {product_edge_id: x[product_edge_id].x for product_edge_id in solver_data.product_edge_ids if x[product_edge_id].x>0}
    v_count = {e: y[e].x for e in data.edges}
    obj_val = model.objVal
    return flow, v_count ,obj_val
  else:
    return None, None, None

In [50]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/10_nodes/')

Result: 1105792110.5968

Solution time: 19.771950244903564



In [51]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/20_nodes/')

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [ ]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/50_nodes/')

In [ ]:
solve_optim_MCF(90, '/content/drive/My Drive/Colab Notebooks/MCF/140_nodes/')